In [15]:
import pandas as pd
import numpy as np
import pyswarms as ps

df = pd.read_csv('cardio_cleaned.csv')
df.head()

gender  cholesterol  gluc  smoke  alco  active  cardio  bmicat  agecat  \
0       1            1     1      0     0       1       0     2.0     5.0   
1       0            3     1      0     0       1       1     4.0     6.0   
2       0            3     1      0     0       0       1     2.0     5.0   
3       1            1     1      0     0       1       1     3.0     4.0   
4       0            1     1      0     0       0       0     2.0     4.0   

   hyper_tension_gr  
0                 1  
1                 2  
2                 1  
3                 2  
4                 1

In [2]:
X = df.copy()
y = X.cardio
X.drop(columns="cardio",inplace=True)
X.shape,y.shape

((66826, 9), (66826,))

In [5]:
from sklearn.model_selection import StratifiedKFold, train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3,random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((46778, 9), (20048, 9), (46778,), (20048,))

In [6]:
X_train = X_train.values
y_train = y_train.values
X_train.shape,y_train.shape

((46778, 9), (46778,))

# Particles Swarm Optimization

In [7]:
n_inputs = X_train.shape[1]
n_hidden = 20
n_classes = 2
num_samples = len(X_train)
iteration = 1000

In [10]:
a = n_inputs*n_hidden
b = a+n_hidden
c = b+(n_hidden*n_classes)
d = c+n_classes

In [11]:
def logits_function(p):
    """ Calculate roll-back the weights and biases

    Inputs
    ------
    p: np.ndarray
        The dimensions should include an unrolled version of the
        weights and biases.

    Returns
    -------
    numpy.ndarray of logits for layer 2

    """
    # Roll-back the weights and biases
    W1 = p[0:a].reshape((n_inputs,n_hidden))
    b1 = p[a:b].reshape((n_hidden,))
    W2 = p[b:c].reshape((n_hidden,n_classes))
    b2 = p[c:d].reshape((n_classes,))

    # Perform forward propagation
    z1 = X_train.dot(W1) + b1  # Pre-activation in Layer 1
    a1 = np.tanh(z1)     # Activation in Layer 1
    logits = a1.dot(W2) + b2 # Pre-activation in Layer 2
    return logits          # Logits for Layer 2


In [12]:
# Forward propagation
def forward_prop(params):
    """Forward propagation as objective function

    This computes for the forward propagation of the neural network, as
    well as the loss.

    Inputs
    ------
    params: np.ndarray
        The dimensions should include an unrolled version of the
        weights and biases.

    Returns
    -------
    float
        The computed negative log-likelihood loss given the parameters
    """

    logits = logits_function(params)

    # Compute for the softmax of the logits
    exp_scores = np.exp(logits)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    # Compute for the negative log likelihood

    corect_logprobs = -np.log(probs[range(num_samples), y_train])
    loss = np.sum(corect_logprobs) / num_samples

    return loss

In [13]:
def f(x):
    """Higher-level method to do forward_prop in the
    whole swarm.

    Inputs
    ------
    x: numpy.ndarray of shape (n_particles, dimensions)
        The swarm that will perform the search

    Returns
    -------
    numpy.ndarray of shape (n_particles, )
        The computed loss for each particle
    """
    n_particles = x.shape[0]
    j = [forward_prop(x[i]) for i in range(n_particles)]
    return np.array(j)

In [16]:
%%time
# Initialize swarm
options = {'c1': 0.8, 'c2': 0.6, 'w':0.9}

# Call instance of PSO
dimensions = (n_inputs * n_hidden) + (n_hidden * n_classes) + n_hidden + n_classes
optimizer = ps.single.GlobalBestPSO(n_particles=100, dimensions=dimensions, options=options)

# Perform optimization
cost, pos = optimizer.optimize(f, iters=iteration)

2024-09-29 20:58:39,322 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.8, 'c2': 0.6, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=0.587
2024-09-29 21:35:55,557 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.5874931325147155, best pos: [ 2.25527013e+00  6.49994420e-01  8.08382498e-01  5.42450623e-01
  7.07049456e-01  9.89214529e-01  5.41315981e-01 -7.80156041e-01
  4.32904957e-01  5.04146149e-01  3.48838436e-01  2.23498737e-01
  6.52699985e-01  1.56662620e+00  6.78252700e-03  3.44388894e-02
  8.40523841e-01  3.71145412e-01  8.45153730e-01  1.46040917e-01
  5.93995751e-01  1.63987044e+00  4.03666243e-01  1.87645093e-01
  7.18463297e-01  2.08262832e-01  2.18838392e+00  1.17520468e+00
 -2.91478305e+00  1.27042931e+00 -2.82904287e-01  3.66244513e-01
  1.00579602e+00  2.45445350e-01  8.14356851e-01 -3.96095428e-01
  9.33616134e-01 -1.94871162e+00  7.57697924e-01  6.12975991e-01
  1.16427959e+00  4.66925

CPU times: total: 13min 59s
Wall time: 37min 16s


In [17]:
def predict(pos):
    """
    Use the trained weights to perform class predictions.

    Inputs
    ------
    pos: numpy.ndarray
        Position matrix found by the swarm. Will be rolled
        into weights and biases.
    """
    logits = logits_function(pos)
    y_pred = np.argmax(logits, axis=1)
    return y_pred

In [18]:
(predict(pos) == y_train).mean()

np.float64(0.7002650818760956)

In [19]:
from pyswarms.utils.plotters.formatters import Mesher
from pyswarms.utils.plotters import (plot_cost_history, plot_contour, plot_surface)
import matplotlib.pyplot as plt
plot_cost_history(cost_history=optimizer.cost_history)
plt.show()

In [21]:
optimizer.cost_history[-1]

np.float64(0.5874931325147155)